In [1]:
import os, cv2
import operator
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
train_df = pd.read_csv("./train/train.csv")
test_df = pd.read_csv("./test/imagenames.csv")
test_df

,id
0,IMG4287_3
1,IMG4288_5
2,IMG4289_5
3,IMG4290_4
4,IMG4291_5
...,...
1195,IMG5482_1
1196,IMG5483_2
1197,IMG5484_4
1198,IMG5485_3


In [ ]:
min_match_count = 4
ratio = 0.6
image_path_test = './test/'
image_path_train = './train/'
all_test_images = []

# Initiate SIFT detector
sift_obj = cv2.SIFT_create()
for test_img_id in test_df['id']:
    test_image = cv2.imread(os.path.join(image_path_test + test_img_id + '.jpg'), cv2.IMREAD_GRAYSCALE)
    # Compute SIFT keypoints and descriptors
    kp_test, des_test = sift_obj.detectAndCompute(test_image, None)
    # FLANN parameters and initialize
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    all_successes = []
    for train_img_id in train_df['id']:
        matchesMask = []
        train_image = cv2.imread(os.path.join(image_path_train + train_img_id + '.jpg'), cv2.IMREAD_GRAYSCALE)
        # Compute SIFT keypoints and descriptors for the test set
        kp_train, des_train = sift_obj.detectAndCompute(train_image, None)
        # Matching descriptor using KNN algorithm
        # Exclude images for which SIFT could not detectAndCompute any descriptors
        if (des_test is not None) and (des_train is not None):
            matches = flann.knnMatch(des_train, des_test, k=2)
            # keeps a list of good matches
            goodMatches = [] 
            for nearest_neighb, second_nearest_neighb in matches:
                # Check the distance ratio between the 1st nearest neighbor and 2nd nearest neighbour
                # Lowe’s ratio test to increase the robustness of the SIFT algorithm and to remove points that are not distinct enough. 
                if nearest_neighb.distance < ratio * second_nearest_neighb.distance:
                    goodMatches.append(nearest_neighb)
                
        if (len(goodMatches) >= min_match_count):
            # Estimate homography between two images
            keypts_train_img = np.zeros((len(goodMatches), 2), dtype=np.float32)
            keypts_test_img = np.zeros((len(goodMatches), 2), dtype=np.float32)
            # Exclude images for which SIFT could not detectAndCompute any keypoints
            if (len(kp_test) != 0) and ((len(kp_train) != 0)):
                keypts_train_img = np.float32([kp_train[match.queryIdx].pt for match in goodMatches]).reshape(-1,1,2)
                keypts_test_img = np.float32([kp_test[match.trainIdx].pt for match in goodMatches]).reshape(-1,1,2)
                # status returns a list of feature points that represent successful matches
                # May require changing the ratio used in the Lowe's ratio test of FLANN matcher and/or the maxIters parameter of the RANSAC
                H, status = cv2.findHomography(keypts_train_img, keypts_test_img, cv2.RANSAC, ransacReprojThreshold = 5, maxIters = 500)
                success = status.ravel().tolist()
                if (success.count(1) > 0):
#                     draw_params = dict(matchColor = (0,255,0), singlePointColor = (255,0,0), matchesMask = success, flags = 2)
#                     success_matches = cv2.drawMatches(train_image, kp_train, test_image, kp_test, goodMatches, None, **draw_params)
                    all_successes.append((test_img_id, train_img_id, success.count(1))) #, success_matches
                else:
                    success = 0
            else:
                success = 0
        # Find the best matched image which has the largest success    
    all_successes.sort(key = operator.itemgetter(2), reverse = True)
    test_img_dict = {}
    if len(all_successes) > 0:
        # Find the ID of best matched image
        best_match = all_successes[0]
        print('Best match:', best_match)
        x_coordiante = train_df[train_df['id'] == best_match[1]]['x']
        y_coordinate = train_df[train_df['id'] == best_match[1]]['y']
        test_img_dict = {'id': best_match[0], 'x': list(x_coordiante)[0] , 'y': list(y_coordinate)[0]}
        #print(test_img_dict)
        # Plotting results
#         for match in best_matches:
#             plt.imshow(match[2])
#             plt.axis('off')
#             plt.show()
    all_test_images.append(test_img_dict)          

Best match: ('IMG4287_3', 'IMG3442_3', 83)
Best match: ('IMG4288_5', 'IMG2755_2', 68)
Best match: ('IMG4289_5', 'IMG3018_5', 48)
Best match: ('IMG4290_4', 'IMG3858_3', 12)
Best match: ('IMG4291_5', 'IMG3207_1', 45)
Best match: ('IMG4292_3', 'IMG3855_1', 35)
Best match: ('IMG4293_3', 'IMG2899_5', 12)
Best match: ('IMG4294_1', 'IMG3713_1', 36)
Best match: ('IMG4295_1', 'IMG2815_1', 164)
Best match: ('IMG4296_1', 'IMG3785_5', 176)
Best match: ('IMG4297_4', 'IMG2831_3', 7)
Best match: ('IMG4298_1', 'IMG2902_3', 40)
Best match: ('IMG4299_3', 'IMG3240_3', 24)
Best match: ('IMG4300_5', 'IMG3043_1', 29)
Best match: ('IMG4301_3', 'IMG3367_5', 5)
Best match: ('IMG4302_2', 'IMG3431_2', 38)
Best match: ('IMG4303_2', 'IMG3473_5', 73)
Best match: ('IMG4304_5', 'IMG3242_5', 35)
Best match: ('IMG4305_5', 'IMG3363_2', 127)
Best match: ('IMG4306_4', 'IMG3071_3', 30)
Best match: ('IMG4307_4', 'IMG4154_4', 39)
Best match: ('IMG4308_3', 'IMG4101_3', 138)
Best match: ('IMG4309_1', 'IMG3367_3', 127)
Best mat